In [1]:
# Importing Dependencies
import pandas as pd
from sqlalchemy import create_engine
import psycopg2 
import io

In [3]:
#Setting path to csv created from api call data
api_data = "./Outputs/active_count.csv"
vg_data = "./Resources/vgsales_clean.csv"
steam_data = "./Resources/steam.csv"

# Converting csv to pandas dataframe and verifying data
player_count_df = pd.read_csv(api_data)
vg_data_df = pd.read_csv(vg_data)
steam_data_df = pd.read_csv(steam_data)

In [4]:
# Defining columns of interest from imported data
count_cols = ["AppID", "Current Players"]

# Dropping columns outside of defined count_cols
count_transformed = player_count_df[count_cols].copy()
count_transformed = count_transformed.rename(columns={"AppID": "appid",
                                                      "Current Players": "current_players"})

# Dropping any potential duplicates to maintain data integrity
count_transformed.drop_duplicates("appid", inplace=True)

# Setting game AppID as index to simplify data location in case multiple get requests are performed and to simplify SQL import
count_transformed.set_index("appid", inplace=True)

# Verifying dataframe columns were transformed correctly
count_transformed.head()

,current_players
appid,
10,14349


In [5]:
# Filling any NA values to avoid pandas conflicts
vg_data_df.fillna(0, inplace=True)

# Setting "Year" column dtype to int instead of float
vg_data_df = vg_data_df.astype({"Year": "int64"})

# Defining columns of interest from imported data
vg_cols = ["Rank", "Name", "Platform", "Year", "Genre", "Publisher", "Global_Sales"]

# Dropping columns outside of defined count_cols
vg_transformed = vg_data_df[vg_cols].copy()

vg_rename = vg_transformed.rename(columns={"Rank": "rank",
                                           "Name": "name",
                                           "Platform": "platform",
                                           "Year": "year",
                                           "Genre": "genre",
                                           "Publisher": "publisher",
                                           "Global_Sales": "global_sales"})

# Verifying dataframe columns were transformed correctly
vg_rename.head()

,rank,name,platform,year,genre,publisher,global_sales
0,1,Wii Sports,Wii,2006,Sports,Nintendo,82.74
1,2,Super Mario Bros.,NES,1985,Platform,Nintendo,40.24
2,3,Mario Kart Wii,Wii,2008,Racing,Nintendo,35.82
3,4,Wii Sports Resort,Wii,2009,Sports,Nintendo,33.00
4,5,Pokemon Red/Pokemon Blue,GB,1996,Role-Playing,Nintendo,31.37


In [6]:
steam_data_df.count()

appid               27075
name                27075
release_date        27075
english             27075
developer           27075
publisher           27075
platforms           27075
required_age        27075
categories          27075
genres              27075
steamspy_tags       27075
achievements        27075
positive_ratings    27075
negative_ratings    27075
average_playtime    27075
median_playtime     27075
owners              27075
price               27075
dtype: int64

In [7]:
steam_data_df = steam_data_df.loc[:, ["appid","name", "release_date", "publisher", "genres", "positive_ratings", "negative_ratings", "average_playtime", "median_playtime", "owners", "price"]]

clean_data_df= steam_data_df.drop_duplicates(keep='first', inplace=False)

clean_data_df.rename(columns={"appid": "appid",
                                        "release_date": "release_date",
                                        "publisher": "publisher",
                                        "genres": "genres",
                                        "positive_ratings": "positive_ratings",
                                        "negative_ratings": "negative_ratings",
                                        "average_playtime": "average_playtime",
                                        "median_playtime": "median_playtime",
                                        "owners": "owners",
                                        "price": "price"}, inplace=True)
    
clean_data_df.head()

,appid,name,release_date,publisher,genres,positive_ratings,negative_ratings,average_playtime,median_playtime,owners,price
0,10,Counter-Strike,2000-11-01,Valve,Action,124534,3339,17612,317,10000000-20000000,7.19
1,20,Team Fortress Classic,1999-04-01,Valve,Action,3318,633,277,62,5000000-10000000,3.99
2,30,Day of Defeat,2003-05-01,Valve,Action,3416,398,187,34,5000000-10000000,3.99
3,40,Deathmatch Classic,2001-06-01,Valve,Action,1273,267,258,184,5000000-10000000,3.99
4,50,Half-Life: Opposing Force,1999-11-01,Valve,Action,5250,288,624,415,5000000-10000000,3.99


In [8]:
# Creating SQL engine
connection_string = "postgres:postgres@localhost:5432/playercount_db"
engine = create_engine(f'postgresql://{connection_string}')

In [9]:
# Verifying tables in database
engine.table_names()

['steam_data1', 'current_players', 'vg_data']

In [10]:
# Importing dataframe data into SQL database
count_transformed.to_sql(name='current_players', con=engine, if_exists='append', index=True)
vg_rename.to_sql(name='vg_data', con=engine, if_exists='append', index=False)
clean_data_df.to_sql(name='steam_data1', con=engine, if_exists='append', index=False)

In [13]:
query_str = open('SQL/query.sql')
query_text = ""

for text in query_str:
    query_text = query_text + text

print(query_text)
query_df = pd.read_sql_query(query_text, con=engine)

query_df

select current_players.appid, 
       current_players.current_players, 
       steam_data1.appid, 
       steam_data1.release_date, 
       steam_data1.name, 
       steam_data1.positive_ratings, 
       steam_data1.negative_ratings, 
       steam_data1.average_playtime, 
       steam_data1.median_playtime, 
       steam_data1.owners, 
       steam_data1.price,
       vg_data.name,
       vg_data.global_sales

from steam_data1
    inner join current_players
        on current_players.appid = steam_data1.appid

    inner join vg_data
        on vg_data.name = steam_data1.name
            order by steam_data1.appid;

-- Joins tables
-- SELECT current_players.appid, current_players.current_players, desired_database_here.desired_column_here, desired_database_here.desired_column_here
-- FROM current_players
-- JOIN "desired_database_here"
-- ON current_players.appid = desired_database_here.desired_column_here;


,appid,current_players,appid,release_date,name,positive_ratings,negative_ratings,average_playtime,median_playtime,owners,price,name,global_sales
0,10,14349,10,2000-11-01,Counter-Strike,124534,3339,17612,317,10000000-20000000,7,Counter-Strike,1.11
